In [1]:
# default_exp signUp

# Sign up

> sign up

In [12]:
#export
import hashlib, uuid, os, logging, sys
import ujson as json
from awsSchema.apigateway import Event,Response
from beartype import beartype
from copy import deepcopy
from pynamodb.models import Model
from pynamodb.attributes import (
    UnicodeAttribute, NumberAttribute, UnicodeSetAttribute, UTCDateTimeAttribute
)

In [3]:
#export
############### Logger for debugging code ##################
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [4]:
#export
########## Error Definitions ##########
class HelperError(Exception): pass
class ParseInputError(HelperError): pass

In [6]:
#hide
os.environ['USERPASSWORDTABLE'] = 'user-password-demo-sallee-master'

In [7]:
#export
################ Setting Globals from Env Vars ################
USERPASSWORDTABLE = os.environ['USERPASSWORDTABLE']

In [8]:
#export
############## Class for accessing DynamoDB #################
class Thread(Model):
    class Meta:
        table_name = USERPASSWORDTABLE
        region = 'ap-southeast-1'

    name = UnicodeAttribute(hash_key=True)
    pHash = UnicodeAttribute(range_key=True)


In [9]:
#export
########## Helper class for main function ##########
EventInput = dict
class H:
    @staticmethod
    def sha256(password):
        return hashlib.sha256(password.encode()).hexdigest()
    
    @staticmethod
    def salt():
        return uuid.uuid4().hex

    @classmethod
    def salted_sha256(cls, password, salt=''):
        if salt == '':
            salt = cls.salt()
        return f'{cls.sha256(salt + password)}:{salt}' if salt is cls.salted_sha256.__defaults__[0] else cls.sha256(salt + password)
    
    @staticmethod
    def add_user_to_table(username, hash):
        userTable = os.environ['USERPASSWORDTABLE']
        threadItem = Thread(username, hash)
        threadItem.save()
        
    
    @staticmethod
    @beartype
    def parseInput(event: EventInput) -> tuple:
        '''
        returns path and filetype arguments from input
        '''
        body = Event.parseBody(deepcopy(event))
        try:
            username = body['username']
        except KeyError:
            logger.error('username is not in body')
            raise ParseInputError('username is not in body')

        try:
            password = body['password']
        except KeyError:
            logger.error('password is not in body')
            raise ParseInputError('password is not in body')

        return username, password

In [10]:
#export
def signUp(event, *args):
  
  evtCpy = deepcopy(event)
  logger.info(f'Event :: {evtCpy}')

  username, password = H.parseInput(evtCpy)
  # Take this away before using, it isn't a good idea to save the username and pw in logs
  logger.info(f"Username :: {username}\npassword :: {password}")

  hashedPw = H.salted_sha256(password)
  logger.info(f'Hashed Pass :: {hashedPw}')
  
  return Response.returnSuccess("this is sign up function")

In [16]:
sampleInput = {'username' : 'Samuel', 'password' : '1234'}
sampleEvent = Event.getInput(body=sampleInput)
print(json.dumps(sampleEvent))


{"body":"{\"username\":\"Samuel\",\"password\":\"1234\"}","httpMethod":null,"multiValueHeaders":null,"multiValueQueryStringParameters":null,"path":null,"pathParameters":null,"queryStringParameters":{},"requestContext":null,"headers":{},"statusCode":200,"isBase64Encoded":false}


In [17]:
signUp(sampleEvent)

Event :: {'body': '{"username":"Samuel","password":"1234"}', 'httpMethod': None, 'multiValueHeaders': None, 'multiValueQueryStringParameters': None, 'path': None, 'pathParameters': None, 'queryStringParameters': {}, 'requestContext': None, 'headers': {}, 'statusCode': 200, 'isBase64Encoded': False}
Username :: Samuel
password :: 1234
Hashed Pass :: 334edd4aff174f590bedca6cc540b290d4de6b2e68d2070d1f475135ce64b07b


{'body': '"this is sign up function"',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
#hide
# import src.setEnv

In [ ]:
#export
# from beartype import beartype
# from awsSchema.apigateway import Response, Event
# from dataclasses import dataclass
# from dataclasses_json import dataclass_json
# from copy import deepcopy

In [ ]:
#export
# from src import passwordHelper as PH
# from src.passwordTable import UserPasswordTable

## User input class

In [ ]:
#export
# @dataclass_json
# @dataclass
# class User:
#   username:str
#   password:str
    
#   @property
#   def passwordHash(self):
#     return PH.hashPassword(self.password)
  
#   def save(self):
#     table = UserPasswordTable(
#       username = self.username,
#       passwordHash = self.passwordHash
#     )
#     table.save()
  

In [ ]:
# SetUpUsername(username = '123', password='123')
# sun = User.from_dict( {'username': 'nic', 'password':'123'})
# sun.passwordHash
# sun.save()

## helper functions

In [ ]:
#export
# class H:
#   class ParseInputError(Exception): pass
#   class SavingUserError(Exception): pass
#   @classmethod
#   @beartype
#   def parseInput(cls,event:dict)->User:
#     try:
#       user = Event.parseDataClass(User,deepcopy(event))
#       return user
#     except Exception as e:
#       raise cls.ParseInputError(e)
  
#   @classmethod
#   @beartype
#   def save(cls, user:User)->bool:
#     try:
#       user.save()
#       return True
#     except Exception as e:
#       raise cls.SavingUserError(e)
      
    
      

## main lambda handler

In [ ]:
#export
# def signUp(event, *args):
#   try:
#     user = H.parseInput(event)
#     H.save(user)
#     return Response.returnSuccess()
#   except H.SavingUserError as e:
#     return Response.returnError(f'failed saving user {e}')
#   except Exception as e:
#     return Response.returnError(f'unknown error {e}')

In [ ]:
# input_ = {'username': 'nic1', 'password':'123'}
# event = Event.getInput(body = input_)
# signUp(event)